In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [2]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

master_dir  = '/Users/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/monte_carlo/'
master_file = 'master.yaml'

with open(f'{master_dir}{master_file}') as file:
    master = yaml.safe_load(file)
       

In [3]:
# list the names of facilities included in our study
facility_names = master['categories']['facility_names']
facility_names

['BP Cherry Point Refinery',
 'Shell Puget Sound Refinery',
 'Shell Oil LP Seattle Distribution Terminal',
 'Maxum (Rainer Petroleum)',
 'Tidewater Snake River Terminal',
 'Nustar Energy Tacoma',
 'SeaPort Sound Terminal',
 'Tesoro Vancouver Terminal',
 'Phillips 66 Ferndale Refinery',
 'Phillips 66 Tacoma Terminal',
 'Andeavor Anacortes Refinery (formerly Tesoro)',
 'Tesoro Port Angeles Terminal',
 'U.S. Oil & Refining',
 'US Oil Tacoma ',
 'Naval Air Station Whidbey Island (NASWI)',
 'NAVSUP Manchester',
 'Alon Asphalt Company (Paramount Petroleum)',
 'Kinder Morgan Liquids Terminal - Harbor Island',
 'Nustar Energy Tacoma',
 'Tesoro Pasco Terminal',
 'REG Grays Harbor, LLC',
 'Tidewater Vancouver Terminal',
 'TLP',
 'TLP Management Services LLC (TMS)']

In [4]:
# Import columns are: (G) Deliverer, (H) Receiver, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")


In [5]:
[nrows,ncols] = df.shape

#### Create dictionary structure

In [79]:
origin = {}
destination = {}

for rows in range(nrows):
  
    # Create dictionary structure for destination
    if df.Receiver[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Receiver[rows] == 'TLP' or 
             df.Receiver[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'
        elif df.Receiver[rows] == 'U.S. Oil & Refining' or 'US Oil' in df.Receiver[rows]:
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[rows]}'
        
        # create dictionaries for each location     
        destination[f'{label}'] = {}
        destination[f'{label}']['total_gallons'] = 0
        destination[f'{label}']['number_of_transfers'] = 0

    # Create dictionary structure for origin
    if df.Deliverer[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Deliverer[rows] == 'TLP' or 
            df.Deliverer[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'      
        elif (df.Deliverer[rows] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[rows]
             ):
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[rows]}'
        
        # create dictionaries for each location     
        origin[f'{label}'] = {}
        origin[f'{label}']['total_gallons'] = 0
        origin[f'{label}']['number_of_transfers'] = 0

# Add accounting placeholders total oil transferred in our analysis transfers
origin['total_gallons']=0
destination['total_gallons']=0
# Add accounting placeholders for total amount of oil missed in our origin/destination 
origin['missed_gallon']=0
destination['missed_gallons']=0

#### Sort volume and transfers by location

In [80]:
the_others_in = pd.DataFrame(columns = df.columns)
the_others_out = pd.DataFrame(columns = df.columns)

for row in range(nrows):
    
    # Catalogue fuel imports (destination traffic)
    if (df.Receiver[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):        
        if (df.Receiver[row] == 'TLP' or 
            df.Receiver[row] == 'TLP Management Services LLC (TMS)'
           ):            
            label = 'TLP Management Services LLC (TMS)'            
        elif (df.Receiver[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Receiver[row]
             ):            
            label = 'U.S. Oil & Refining'            
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[row]}'
        
        # add up fuel and transfers to the locations included in our study
        destination[f'{label}']['total_gallons'] += df.TransferQtyInGallon[row].item()
        destination[f'{label}']['number_of_transfers'] += 1
        
        # add up all fuel received as cargo to the terminal locations used in our study
        destination['total_gallons'] += df.TransferQtyInGallon[row].item()
           
    # Catalogue fuel exports (destination traffic)
    elif (df.Deliverer[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):        
        if (df.Deliverer[row] == 'TLP' or 
            df.Deliverer[row] == 'TLP Management Services LLC (TMS)'
           ):            
            label = 'TLP Management Services LLC (TMS)'            
        elif (df.Deliverer[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[row]
             ):            
            label = 'U.S. Oil & Refining'            
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[row]}'
        
        # add up fuel and transfers from the locations included in our study
        origin[f'{label}']['total_gallons'] += df.TransferQtyInGallon[row].item()
        origin[f'{label}']['number_of_transfers'] += 1
        
        # add up all fuel delivered as cargo from the terminal locations used in our study
        origin['total_gallons'] += df.TransferQtyInGallon[row].item()
              
    elif (df.TransferType[row] == 'Cargo' and
         df.DelivererTypeDescription[row] == 'Facility'):
        the_others_out = the_others_out.append( df.iloc[row] )
    elif (df.TransferType[row] == 'Cargo' and
         df.ReceiverTypeDescription[row] == 'Facility'):
        the_others_in = the_others_in.append( df.iloc[row] )


In [81]:
the_others_in.reset_index( drop=True, inplace=True )
the_others_out.reset_index( drop=True, inplace=True )

In [87]:
origin['missed_gallons'] = the_others_in.TransferQtyInGallon.sum()
destination['missed_gallons'] = the_others_out.TransferQtyInGallon.sum()

total_out = origin['missed_gallons'] + origin['total_gallons']
total_in  = destination['missed_gallons'] + destination['total_gallons']

In [91]:
print(f'Percentage of outgoing fuel transfer missed by our terminal selection: {100*origin["missed_gallons"]/total_out:4.2f}')
print(f'Percentage of incoming fuel transfers missed by our terminal selection: {100*destination["missed_gallons"]/total_in:4.2f}')

Percentage of outgoing fuel transfer missed by our terminal selection: 2.76
Percentage of incomming fuel transfers missed by our terminal selection: 0.60


### create arrays of terminal names that we missed in our analysis
(toward quantifying quantity by missed terminal)

In [60]:
[rows_in, cols_in] = the_others_in.shape
other_facilities_in = {}

for row in range(rows_in):
    if row == 0: 
        other_facilities_in['name'] = [the_others_in.Receiver[row]]
    else:
        if the_others_in.Receiver[row] not in other_facilities_in['name']:
            other_facilities_in['name'].append(the_others_in.Receiver[row])
      
[rows_out, cols_out] = the_others_out.shape
other_facilities_out = {}
for row in range(rows_out):
    if row == 0:         
        other_facilities_out['name'] = [the_others_out.Deliverer[row]]
    else:     
        if the_others_out.Deliverer[row] not in other_facilities_out['name']:
            other_facilities_out['name'].append(the_others_out.Deliverer[row])
              

In [61]:
other_facilities_in['name']

['Tidewater Umatilla Terminal',
 'Naval FISC Manchester Fuel Depot (Mobile)',
 'CHS PRIMELAND',
 'Alon Point Wells',
 'Nustar Energy Vancouver',
 'CHS PRIMELAND / MEGREGOR ',
 'Andeavor Port Angeles',
 'NEW ENDEAVOR',
 'Ross Lake Resort',
 'Shell',
 'EVER LYRIC',
 'Seaport Seattle Terminal ']

In [62]:
other_facilities_out['name']

['Pacific Ethanol',
 'Naval FISC Manchester Fuel Depot (Mobile)',
 'BP West Coast Arco Seattle Terminal',
 'Clean Harbors Inc.',
 'Nustar Energy Vancouver']

### Calculate percentage of exported oil not accounted for in our terminal analysis
(I decided not to bother with this b/c the quantity of oil is small) 


In [67]:
for terminals in other_facilities_in['name']:
    other_facilities_in[f'{terminals}'] = the_others_in[the_others_in['Receiver']==f'{terminals}'].TransferQtyInGallon.sum()

In [97]:
other_facilities_in

{'name': ['Tidewater Umatilla Terminal',
  'Naval FISC Manchester Fuel Depot (Mobile)',
  'CHS PRIMELAND',
  'Alon Point Wells',
  'Nustar Energy Vancouver',
  'CHS PRIMELAND / MEGREGOR ',
  'Andeavor Port Angeles',
  'NEW ENDEAVOR',
  'Ross Lake Resort',
  'Shell',
  'EVER LYRIC',
  'Seaport Seattle Terminal '],
 'Tidewater Umatilla Terminal': 54506547,
 'Naval FISC Manchester Fuel Depot (Mobile)': 28177800,
 'CHS PRIMELAND': 393684,
 'Alon Point Wells': 1449000,
 'Nustar Energy Vancouver': 18144000,
 'CHS PRIMELAND / MEGREGOR ': 984210,
 'Andeavor Port Angeles': 2624560,
 'NEW ENDEAVOR': 850080,
 'Ross Lake Resort': 7000,
 'Shell': 13020000,
 'EVER LYRIC': 1082631,
 'Seaport Seattle Terminal ': 3360000}

In [98]:
other_facilities_in['Seaport Seattle Terminal ']/total_in

0.0005962584991065779

In [99]:
other_facilities_in['Shell']/total_in

0.0023105016840379894

In [92]:
other_facilities_out

{'name': ['Pacific Ethanol',
  'Naval FISC Manchester Fuel Depot (Mobile)',
  'BP West Coast Arco Seattle Terminal',
  'Clean Harbors Inc.',
  'Nustar Energy Vancouver']}